In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import time

# ChromeDriver setup
chrome_driver_path = "C:\\chromedriver-win64\\chromedriver.exe"
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Uncomment to hide browser
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

# Go to the CEYPETCO historical prices page
url = "https://ceypetco.gov.lk/historical-prices/"
driver.get(url)
time.sleep(2)

lad_prices = []

def extract_from_current_page():
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")

    for row in rows[1:]:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) >= 4:
            date_str = cells[0].text.strip()
            lad_price = cells[3].text.strip()

            try:
                date_obj = datetime.strptime(date_str, "%d.%m.%Y")
                if date_obj >= datetime(2024, 1, 1):
                    lad_prices.append({
                        'date': date_obj.strftime("%Y-%m-%d"),
                        'lad_price': lad_price
                    })
            except ValueError:
                continue

# Find all page numbers
pagination = driver.find_elements(By.XPATH, "//a[@data-page]")
page_numbers = sorted(set(int(p.get_attribute("data-page")) for p in pagination))

for page in page_numbers:
    print(f"Scraping page {page}...")
    try:
        page_link = driver.find_element(By.XPATH, f"//a[@data-page='{page}']")
        driver.execute_script("arguments[0].click();", page_link)
        time.sleep(2)
        extract_from_current_page()
    except NoSuchElementException:
        print(f"Page {page} not found, skipping.")

driver.quit()

# Sort data
lad_prices.sort(key=lambda x: x['date'], reverse=True)

# Save to CSV
csv_filename = "feul_price.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["date", "lad_price"])
    writer.writeheader()
    writer.writerows(lad_prices)

print(f"\n✅ Data saved to '{csv_filename}'")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...

✅ Data saved to 'feul_price.csv'
